In [1]:
!nvidia-smi

Wed Dec 11 01:36:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━

In [2]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1
Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
Found existing installation: accelerate 1.2.0
Uninstalling accelerate-1.2.0:
  Successfully uninstalled accelerate-1.2.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
  Using cached accelerate-1.2.0-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 43.2 MB/s eta 0:00:00
Using cached accelerate-1.2.0-py3-none-any.whl (336 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 65.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully un

In [3]:
!pip install --upgrade datasets

In [4]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.7 MB/s eta 0:00:00


In [5]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import evaluate
import matplotlib.pyplot as plt
import pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [8]:
#download and unzip the data

!wget https://github.com/thedavidmighty/text-summarizer/raw/refs/heads/main/data/text-summariser-data.zip

--2024-12-11 01:40:07--  https://github.com/thedavidmighty/text-summarizer/raw/refs/heads/main/data/text-summariser-data.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/thedavidmighty/text-summarizer/refs/heads/main/data/text-summariser-data.zip [following]
--2024-12-11 01:40:07--  https://raw.githubusercontent.com/thedavidmighty/text-summarizer/refs/heads/main/data/text-summariser-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘text-summariser-data.zip’

text-summariser-dat 100%[===================>]   7.54M  --.-KB/s    in 0.07s   

202

In [9]:
!unzip text-summariser-data.zip

Archive:  text-summariser-data.zip
  inflating: samsum-test.csv         
  inflating: samsum-train.csv        
  inflating: samsum-validation.csv   
   creating: samsum_dataset/
 extracting: samsum_dataset/dataset_dict.json  
   creating: samsum_dataset/test/
  inflating: samsum_dataset/test/data-00000-of-00001.arrow  
  inflating: samsum_dataset/test/dataset_info.json  
  inflating: samsum_dataset/test/state.json  
   creating: samsum_dataset/train/
  inflating: samsum_dataset/train/data-00000-of-00001.arrow  
  inflating: samsum_dataset/train/dataset_info.json  
  inflating: samsum_dataset/train/state.json  
   creating: samsum_dataset/validation/
  inflating: samsum_dataset/validation/data-00000-of-00001.arrow  
  inflating: samsum_dataset/validation/dataset_info.json  
  inflating: samsum_dataset/validation/state.json  


In [10]:
dataset_samsum = load_from_disk("samsum_dataset")
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [11]:
split_len = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"Split length: {split_len}")
print(f"features: {dataset_samsum['train'].column_names}")
print("\nhere is the dialogue:")

print(dataset_samsum["train"][1]["dialogue"])

print("\n Here is the Summary:")

print(dataset_samsum["train"][1]["summary"])

Split length: [14732, 819, 818]
features: ['id', 'dialogue', 'summary']

here is the dialogue:
Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great

 Here is the Summary:
Olivia and Olivier are voting for liberals in this election. 


In [12]:
#now we need to convert the statements into features that the model can understand, i.e. tokenisation

def convert_samples_to_features(sample_batch):
  input_encodings = tokenizer(sample_batch["dialogue"], max_length=1024, truncation=True)

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(sample_batch["summary"], max_length=128, truncation=True)

  return {
      "input_ids": input_encodings["input_ids"],
      "attention_mask": input_encodings["attention_mask"],
      "labels":  target_encodings["input_ids"]
  }

In [13]:
#Now i would need to apply the tokenizarion function on my data

dataset_samsum_tokenized = dataset_samsum.map(convert_samples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [14]:
print(dataset_samsum_tokenized["train"])
print(dataset_samsum_tokenized["test"])

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})
Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 819
})


In [15]:
#Now we have tokenized to how the model understands the inputs , we can start traning the model for our specify dataset

from transformers import DataCollatorForSeq2Seq

seq_2_seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

In [16]:


from transformers import TrainingArguments, Trainer

training_params = TrainingArguments(output_dir = "pegasus-samsum", num_train_epochs = 1, warmup_steps = 500,
                                    per_device_train_batch_size = 1, per_device_eval_batch_size = 1,
                                    weight_decay = 0.01, logging_steps = 10,
                                    evaluation_strategy = "steps", eval_steps = 500, save_steps = 1e6,
                                    gradient_accumulation_steps = 16
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer = Trainer(model = model_pegasus, args = training_params,
                  tokenizer = tokenizer, data_collator = seq_2_seq_data_collator,
                  train_dataset = dataset_samsum_tokenized["train"],
                  eval_dataset = dataset_samsum_tokenized["validation"])

<ipython-input-17-0691b5dd8a24>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model = model_pegasus, args = training_params,


In [18]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
500,1.659700,1.484200


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=920, training_loss=1.825385618209839, metrics={'train_runtime': 2795.1322, 'train_samples_per_second': 5.271, 'train_steps_per_second': 0.329, 'total_flos': 5528248038285312.0, 'train_loss': 1.825385618209839, 'epoch': 0.9991854466467553})

In [20]:
from evaluate import load
metric = load("rouge")

In [22]:
from tqdm import tqdm
from evaluate import load

# Load the ROUGE metric
metric = load("rouge")

def generate_batch_sized_bits(list_of_items, batch_size):
    for i in range(0, len(list_of_items), batch_size):
        yield list_of_items[i : i + batch_size]

def calculate_metrics_on_test(dataset, metric, model, tokenizer,
                              batch_size=16, device="cuda",
                              column_text="article",
                              column_summary="highlights"):
    # Generate batches
    article_batches = list(generate_batch_sized_bits(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_bits(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)
    ):
        # Tokenize the input articles
        inputs = tokenizer(
            article_batch, max_length=1024, truncation=True,
            padding="max_length", return_tensors="pt"
        )
        # Move inputs to device
        inputs = {key: val.to(device) for key, val in inputs.items()}

        # Generate summaries
        encoded_summaries = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            length_penalty=0.8,
            num_beams=8,
            max_length=128
        )

        # Decode generated summaries
        decoded_summaries = [
            tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for s in encoded_summaries
        ]
        # Decode target summaries
        decoded_summaries = [d.replace(" ", " ") for d in decoded_summaries]

        # Add predictions and references to the metric
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Compute the ROUGE scores
    score = metric.compute()
    return score


In [ ]:
# #Evaluate

# def generate_batch_sized_bits(list_of_items, batch_size):
#   for i in range(0, len(list_of_items), batch_size):
#     yield list_of_items[i : i + batch_size]

# def calculate_metrics_on_test(dataset, metric, model, tokenizer,
#                               batch_size = 16, device = device,
#                               column_text = "article",
#                               column_summary = "highlights"):
#   article_batches = list(generate_batch_sized_bits(dataset[column_text], batch_size))
#   target_batches = list(generate_batch_sized_bits(dataset[column_summary], batch_size))


#   for article_batch, target_batch in tqdm(
#       zip(article_batches, target_batches), total = len(article_batches)):

#       inputs = tokenizer(article_batch, max_length = 1024, truncation=True,
#                          padding= "max_length", return_tensors = "pt")
#       encoded_summaries = model.generate(input_ids = inputs["input_ids"].to(device),
#                                  attention_mask = inputs["attention_mask"].to(device),
#                                  length_penalty = 0.8, num_beams = 8, max_length = 128)

#       #now we need to decode the generated text, replacing the tokens with eth decoded texts refernce to metrics
#       decoded_summaries = [tokenizer.decode(s, skip_special_tokens = True,
#                                             clean_up_tokenization_spaces = True)
#               for s in encoded_summaries]
#       decoded_summaries = [d.replace(" ", " ") for d in decoded_summaries]

#       metric.add_batch(predictions = decoded_summaries, references = target_batch)

# #Finally compute and return the Rouge scores

# score = metric.compute()
# return score


In [24]:
rouge_name = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load("rouge")

In [27]:
# Generate the scores
score = calculate_metrics_on_test(
    dataset_samsum["test"],
    rouge_metric,
    trainer.model,
    tokenizer,
    batch_size=2,
    column_text="dialogue",
    column_summary="summary"
)

# Extract ROUGE scores into a dictionary
rouge_dict = {rn: score[rn] for rn in score.keys()}  # Directly access the scalar scores

# Create a DataFrame
import pandas as pd
pd.DataFrame([rouge_dict], index=["pegasus"])

100%|██████████| 410/410 [13:25<00:00,  1.96s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.427589,0.197995,0.339748,0.339889


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['rouge1']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'rouge1'}, axis=1)
              .sort_values('rouge1', ascending=True))
  xs = counted['rouge1']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('rouge1', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('rouge1')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['rouge2']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'rouge2'}, axis=1)
              .sort_values('rouge2', ascending=True))
  xs = counted['rouge2']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('rouge2', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('rouge2')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['rougeL']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'rougeL'}, axis=1)
              .sort_values('rougeL', ascending=True))
  xs = counted['rougeL']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('rougeL', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('rougeL')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['rougeLsum']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'rougeLsum'}, axis=1)
              .sort_values('rougeLsum', ascending=True))
  xs = counted['rougeLsum']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('rougeLsum', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('rougeLsum')
_ = plt.ylabel('count()')

In [35]:
model_pegasus.save_pretrained("pegasus_samsum_model_1")

In [29]:
#Saving tokenizer
tokenizer.save_pretrained("tokenizer_1")

('tokenizer_1/tokenizer_config.json',
 'tokenizer_1/special_tokens_map.json',
 'tokenizer_1/spiece.model',
 'tokenizer_1/added_tokens.json',
 'tokenizer_1/tokenizer.json')

In [30]:
#Loading the Model

In [31]:
tokenizer = AutoTokenizer.from_pretrained("tokenizer_1")

In [37]:
#Making prediction

gen_kwargs = {"length_penalty":0.8, "num_beams":9, "max_length":100}

sample_text = dataset_samsum["test"][2]["dialogue"]

reference = dataset_samsum["test"][2]["summary"]

pipe = pipeline("summarization", model = "pegasus-samsum-model_1", tokenizer = tokenizer)

print("Dialogue")
print(sample_text)


print("\nReference Summary")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Device set to use cuda:0


Dialogue
Lenny: Babe, can you help me with something?
Bob: Sure, what's up?
Lenny: Which one should I pick?
Bob: Send me photos
Lenny:  <file_photo>
Lenny:  <file_photo>
Lenny:  <file_photo>
Bob: I like the first ones best
Lenny: But I already have purple trousers. Does it make sense to have two pairs?
Bob: I have four black pairs :D :D
Lenny: yeah, but shouldn't I pick a different color?
Bob: what matters is what you'll give you the most outfit options
Lenny: So I guess I'll buy the first or the third pair then
Bob: Pick the best quality then
Lenny: ur right, thx
Bob: no prob :)

Reference Summary
Lenny can't decide which trousers to buy. Bob advised Lenny on that topic. Lenny goes with Bob's advice to pick the trousers that are of best quality.

Model Summary:
Lenny wants Bob to send him photos of purple trousers. Lenny already has purple trousers. Bob has four black trousers. Lenny will buy the first or the third pair.
